In [41]:
import cv2
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
import matplotlib.pyplot as plt
import random
import os
from PIL import Image

def img_res(label,txt,im):
    total=[] # 최종 정렬된 번호 입력
    re=[] # 정렬되기 전 라벨 입력
    bbox=[] # 라벨 입력
    
    #image 불러오기
    img = cv2.imread(im)
    
    #라벨 불러오기
    gt = np.loadtxt(txt, dtype = float, delimiter = ' ')
    
    #image size
    img_x, img_y = img.shape[1],img.shape[0]
    
    #라벨이 없을 경우, 그대로 밖으로
    if len(gt)==0:
        return ""
    
    #1차원일 경우, 그대로 gt에 넣어준다.
    if gt.ndim==1:
        gt=np.array([gt])
    
    
    input_img = img[np.newaxis, :, :, :]
    
    #라벨을 bounding box로 변환 후 리스트에 추가
    for i in range(gt.shape[0]):
        bbox.append(ia.BoundingBox(x1 = gt[i][1]*img_x-(gt[i][3]*img_x)/2, y1 = gt[i][2]*img_y-(gt[i][4]*img_y)/2, x2 = gt[i][1]*img_x+(gt[i][3]*img_x)/2, y2 = gt[i][2]*img_y+(gt[i][4]*img_y)/2))

    #출력 이미지와 bbox를 가져오기
    output_img, output_bbox = iaa.Affine(scale = 1)(images = input_img, bounding_boxes = bbox)
    
    #해당 bbox는 결과만 나오지 않고 텍스트 형식으로 나오기 때문에, 이를 바꿔준다.
    for k in range(gt.shape[0]):
        box_list=[]
        box_list2=[]
        result = ' '.join(str(s) for s in [output_bbox[k]])
        box_split = result.split(',')
        box_list.append(box_split[0].split('='))
        box_list.append(box_split[1].split('='))
        box_list.append(box_split[2].split('='))
        box_list.append(box_split[3].split('='))
        box_list2.append((float(box_list[0][1])+float(box_list[2][1]))/2/img_x)
        box_list2.append((float(box_list[1][1])+float(box_list[3][1]))/2/img_y)
        box_list2.append((float(box_list[2][1])-float(box_list[0][1]))/img_x)
        box_list2.append((float(box_list[3][1])-float(box_list[1][1]))/img_y)
        re.append([int(gt[k][0]),float(box_list2[0]),float(box_list2[1]),float(box_list2[2]),float(box_list2[3])])
    
    #bbox들을 x축 순서로 정리해준다.
    for u in range(len(re)-1):
        for t in range(len(re)-1-u):
            if re[t][1]>re[t+1][1]:
                k=re[t]
                re[t]=re[t+1]
                re[t+1]=k
    
    #total이라는 리스트에 Class값만 다시 추가해준다.
    for y in range(len(re)):
        total.append(re[y][0])
        
    #Class에 해당하는 label을 가져온다.
    for w in range(len(total)):
        total[w]=label[total[w]]
        
        
        
    return total



root_dir = "C:/scr/darknet/test_dataset_yhg" # 디렉토리

possible_img_extension = ['.jpg', '.jpeg', '.JPG', '.bmp', '.png'] # 이미지 확장자들

fi_name = "re3"

label_list=[]
os.mkdir("C:/scr/darknet" + "/"+ fi_name)
f1 = open("C:/scr/darknet/build/darknet/x64/data/plate_num.txt",'r',encoding='UTF8')
while True:
    line = f1.readline()
    if not line: break
    label_list.append(str(line))

f1.close()

f2 = open("C:/scr/darknet/build/darknet/x64/data/plate_all.txt",'w')

for (root, dirs, files) in os.walk(root_dir):
    if len(files) > 0:
        for file_name in files:
            if os.path.splitext(file_name)[1] in possible_img_extension:
                box_list=[]
                img_path = root + '/' + file_name
                img_path = img_path.replace('\\', '/') # \는 \\로 나타내야함
                box = img_res(label_list,img_path[:-4]+'.txt',img_path)

                if box=="":
                    continue
                
#                f = open("C:/scr/darknet/" + fi_name + "/resort_"+file_name[:-4]+".txt",'w')
                f2.write(file_name+ " ")
                for r in range(len(box)):
                    f.write(box[r].strip())
                f2.write("\n")
                
f2.close()
                
                
    


FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'C:/scr/darknet/re3'

In [15]:

f1.close()